<font style="font-size:96px; font-weight:bolder; color:#0040a0"><img src="http://montage.ipac.caltech.edu/docs/M51_logo.png" alt="M" style="float: left; padding: 25px 30px 25px 0px;" /></font>

<i><b>Montage</b> Montage is an astronomical image toolkit with components for reprojection, background matching, coaddition and visualization of FITS files. It can be used as a set of command-line tools (Linux, OS X and Windows), C library calls (Linux and OS X) and as Python binary extension modules.

The Montage source is written in ANSI-C and code can be downloaded from GitHub ( https://github.com/Caltech-IPAC/Montage ). The Python package can be installed from PyPI ("</i>pip install MontagePy<i>"). The package has no external dependencies. See http://montage.ipac.caltech.edu/ for details on the design and applications of Montage.


# MontagePy.main modules: mDiffFitExec

The Montage modules are generally used as steps in a workflow to create a mosaic of a set of input images.  These steps are: determine the geometry of the mosaic on the sky, reproject the images to a common frame and spatial sampling; rectify the backgrounds to a common level, and coadd the images into a mosaic.  This page illustrates the use of one Montage module, mDiffFitExec, which differences and fits a set of image pairs (image overlaps).

Visit <a href="Mosaic.ipynb">Building a Mosaic with Montage</a> to see how mDiffFitExec is used as part of a workflow to creage a mosaic (or the <a href="Mosaic_oneshot.ipynb"> one shot </a> version if you just want to see the commands).  See the complete list of Montage Notebooks <a href="http://montage.ipac.caltech.edu/MontageNotebooks">here</a>.


In [1]:
from MontagePy.main import mDiffFitExec, mViewer

help(mDiffFitExec)

Help on built-in function mDiffFitExec in module MontagePy.main:

mDiffFitExec(...)
    mDiffFitExec loops over a list of image overlaps, computing then fitting the differences.
    
    Parameters
    ----------
    path : str
        Path to raw image directory.
    tblfile : str
        Table file list of input images.
    template : str
        FITS header file used to define the desired output.
    diffdir : str
        Directory for temporary difference files.
    fitfile : str
        Table file for output difference fit info.
    keepAll : bool, optional
        Flag to keep temporary difference images.
    levelOnly : bool, optional
        Flag to fit level of differences only, not slopes.
    noAreas : bool, optional
        Flag indicating there are no area images.
    debug : int, optional
        Debugging output flag.
    
    
    Returns
    -------
    status : int
        Return status (0: OK, 1:ERROR).
    msg : str
        Return message (for errors).
    count : i

## mDiffFitExec Example

mDiffFitExec is a general utility for generating and fitting a set of image differences. It's most common use is as part of the process of evaluating overlap area in a set of images as part of background rectification. That process consists of taking the output of the overlaps calculation (mOverlaps) and for each overlap pair doing the difference (mDiff), fitting it to determine offset between those two images (mFitplane).

There are two ways to do this in bulk. We can either generate all the differences first, then fit them all or we can loop over the differences, generating and fitting that difference before moving on to the next. This first approach uses more disks space as all the differences have to be stored in between steps. The second can clean up as it goes.

Obviously the second approach is preferable unless you want to keep the differences (possibly for quality evaluation).

mDiffFitExec is an instantiation of the second approach. It runs mDiff on every pair in the input list and then fits it with a plane.

At this point in the processing, we already have a set of "like" images (same projection), and have made a list of all the overlaps (normally using mOverlaps).  mDiffFitExec loops over these differences, generating each difference (in a working directory) and fitting a plane to it.

mDiffFitExec cleans up the working files as it goes.

In [2]:
rtn = mDiffFitExec('M17/projected', 
                   'M17/diffs.tbl', 
                   'M17/M17.hdr',
                   'M17/diffs',
                   'work/M17/fits.tbl')
print(rtn)

{'status': '0', 'count': 128, 'diff_failed': 0, 'fit_failed': 0, 'warning': 0}


The output is a set of planes representing the difference between two original images.  This set will be sent to mBgModel for modelling into a set of corrections to the original images to bring the backgrounds in line.

In [3]:
import numpy as np
import pandas as pd
from astropy.io import ascii

ipactable = ascii.read('work/M17/fits.tbl').to_pandas()

ipactable

,plus,minus,a,b,c,crpix1,crpix2,xmin,xmax,ymin,ymax,xcenter,ycenter,npixel,rms,boxx,boxy,boxwidth,boxheight,boxang
0,0,8,1.268790e-03,4.644410e-04,0.410733,-1711.0,1756.5,1711,1789,-1756,-935,1750.15,-1343.07,52864,7.362550e-01,1749.9,-1346.0,821.2,77.3,-90.1
1,0,9,-2.523660e-03,1.653820e-03,6.969540,-1713.0,988.5,1713,1789,-988,-733,1751.01,-863.07,16232,7.125360e-01,1751.1,-861.0,255.3,76.9,-90.2
2,0,13,2.453210e-03,3.935060e-03,0.277418,-1286.0,986.5,1286,1361,-986,-732,1323.43,-862.04,16218,7.174900e-01,1323.5,-859.5,254.0,75.0,90.0
3,0,14,-2.490360e-03,8.991680e-04,4.436860,-1283.0,1755.5,1283,1361,-1755,-933,1322.70,-1342.62,52649,7.105830e-01,1322.3,-1344.5,822.2,77.3,89.9
4,0,21,-3.303240e-10,5.376120e-09,-1.130130,-1286.0,786.5,1286,1790,-786,-732,1537.86,-759.37,27003,7.587320e-06,1538.0,-759.5,54.0,504.0,90.0
5,0,24,-1.489270e-10,-5.884170e-09,0.561656,-1284.0,1756.5,1284,1787,-1756,-1702,1535.59,-1729.23,27090,7.108770e-06,1535.5,-1729.5,54.0,503.0,90.0
6,0,29,-1.230960e-03,5.115690e-04,-79.055300,-1710.0,994.5,1710,1790,-994,-733,1749.78,-865.65,16584,7.530540e-01,1749.9,-864.0,261.3,79.3,-90.2
7,0,31,2.329150e-03,1.100920e-03,-82.140500,-1708.0,1756.5,1708,1789,-1756,-941,1748.71,-1348.03,53097,7.707710e-01,1748.8,-1349.0,815.2,79.9,89.8
8,1,6,1.756510e-04,-2.708390e-04,-3.833850,426.0,789.5,-426,-351,-789,32,-387.36,-347.45,42063,9.129390e-01,-388.5,-378.5,822.0,75.0,90.0
9,1,11,-2.720230e-03,-5.054520e-04,-0.537130,-2.0,789.5,2,78,-789,31,40.09,-390.72,47921,8.727160e-01,40.0,-379.0,821.0,76.0,90.0


&nbsp;<p/>

## Error Handling

If mDiffFitExec encounters an error, the return structure will just have two elements: a status of 1 ("error") and a message string that tries to diagnose the reason for the error.

For instance, if the user specifies a table that doesn't exist: 


In [4]:
rtn = mDiffFitExec('M17/projected', 
                   'M17/unknown.tbl', 
                   'M17/M17.hdr',
                   'M17/diffs',
                   'work/M17/fits.tbl')
print(rtn)

{'status': '1', 'msg': b'Invalid diffs metadata file: M17/unknown.tbl'}


&nbsp;

## Classic Montage: mDiffFitExec as a Stand-Alone Program


### mDiffFitExec Unix/Windows Command-line Arguments

<p>mDiffFitExec can also be run as a command-line tool in Linux, OS X, and Windows:</p>

<p><tt>
<b>Usage:</b> mDiffFitExec [-d] [-l(evel-only)] [-n(o-areas)] [-p projdir] [-s statusfile] diffs.tbl template.hdr diffdir fits.tbl
</tt></p>
<p>&nbsp;</p>
<p>If you are writing in C/C++, mDiffFitExec can be accessed as a library function:</p>


<pre>
/*-*****************************************************************/
/*                                                                 */
/*  mDiffFitExec                                                   */
/*                                                                 */
/*  This routine combines the mDiff and mFit functionality and     */
/*  optionally discards the difference images as it goes (to       */
/*  minimize the file space needed).   It uses the table of        */
/*  oerlaps found by mOverlaps, running mDiff, then mFitplane      */
/*  on the difference images.  These fits are written to an        */
/*  output file which is then used by mBgModel.                    */
/*                                                                 */
/*   char *path        Path to images to be diffed.                */
/*   char *tblfile     Table file list of images to diff.          */
/*   char *template    FITS header file used to define the desired */
/*                     output.                                     */
/*                                                                 */
/*   char *diffdir     Directory for temporary output diff files.  */
/*   char *fitfile     Table file for output difference fits info. */
/*   int keepAll       Flag to keep temporary diff images.         */
/*   int levelOnly     Flag to fit level of diff only, not slopes. */
/*   int noAreas       Flag indicating there are no area images.   */             
/*   int debug         Debug flag.                                 */
/*                                                                 */
/*******************************************************************/

struct mDiffFitExecReturn *mDiffFitExec(char *path, char *tblfile, char *template, char *diffdir,
                                        char *fitfile, int keepAll, int levelOnly, int noAreas, int debugin)
</pre>
<p><b>Return Structure</b></p>
<pre>
struct mDiffFitExecReturn
{
   int    status;        // Return status (0: OK, 1:ERROR)
   char   msg [1024];    // Return message (for error return)
   char   json[4096];    // Return parameters as JSON string
   int    count;         // Number of differences                
   int    diff_failed;   // Number of differences that failed
   int    fit_failed;    // Number of fits to differences that failed
   int    warning;       // Number of fits to differences that produced warnings
};
</pre>